
"""
CHECK

- The Missing Value Treatement for the rtgp (runtime in group) variables - espcially for races where NONE of the dogs have a runtime

More Variables :

Total Races from current race
- Races in the last 7/14/28/60/90/180/365/730 days 
- Races won in the 7/14/28/60/90/180/365/730 days
- Races placed in the 7/14/28/60/90/180/365/730 days


"""


# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:
DEBUG = True

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


## IMPORTING THE RAW BASE

In [7]:

df.ft_raw = pd.read_csv(f'{fileloc}/raw_base_2022-12-03.csv', parse_dates = True)

print(df.ft_raw.shape)
print(df.ft_raw.columns.values.tolist())



(588017, 33)
['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos']


In [8]:

df_base_sp = df.ft_raw.copy()


### B) Create Variable for Speed Profile

In [9]:

# Sort for Easy QC
df_base_sp.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)


In [10]:

df_base_sp[df_base_sp['@id_DOG'].isin([108390981])]


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position,y_margin,y_pos
0,108390981,4,DOLLY DOES IT,5.0,5,30.4,$11.00,NaN,6.00,0.14,0,0,B3,NaN,26.43,80.0,626271182,92892,M Elphinstone,626271182,10,GREAT NORTHERN,03:14PM,452m,Mixed 4/5,Devonport,05 Jan 21,2021-01-05,11.0,8,4,6.00,4
1,108390981,5,DOLLY DOES IT,4.0,4,30.2,$7.00,NaN,6.00,1.14,0,0,NaN,NaN,26.40,0.0,629797242,92892,M Elphinstone,629797242,10,GREAT NORTHERN,03:09PM,452m,Grade 5,Devonport,12 Jan 21,2021-01-12,7.0,7,5,6.00,5
2,108390981,1,DOLLY DOES IT,5.0,5,30.2,$7.00,NaN,5.25,NaN,0,0,NaN,8.1,26.20,1890.0,629797513,92892,M Elphinstone,629797513,4,SUREPICK,01:27PM,452m,Grade 5,Devonport,19 Jan 21,2021-01-19,7.0,7,1,5.25,1
3,108390981,7,DOLLY DOES IT,1.0,1,30.0,$7.00,NaN,10.50,2.00,0,0,NaN,NaN,26.48,0.0,638810808,92892,M Elphinstone,638810808,8,LADBROKES,02:22PM,452m,Mixed 4/5 Heat,Devonport,16 Feb 21,2021-02-16,7.0,8,7,10.50,7
4,108390981,8,DOLLY DOES IT,8.0,8,29.9,$91.00,NaN,18.50,3.57,0,0,SO,NaN,26.91,0.0,643554706,92892,M Elphinstone,643554706,8,NEW RECRUIT 2012,02:22PM,452m,Grade 4,Devonport,02 Mar 21,2021-03-02,91.0,8,8,18.50,8


In [11]:

# ADDITIONAL VARIABLES
doglevel = ['@id_DOG']

# Distance and Speed in each race
df_base_sp['x_dist'] = df_base_sp['Distance'].apply(lambda x : x.replace('m','')).astype(int)
df_base_sp['speed'] = (df_base_sp['x_dist'] / df_base_sp['y_runtime'])*1000

# Flag for getting a total race count
df_base_sp['f_1'] = 1

# Total Races
df_base_sp['x_races_tot'] = df_base_sp.groupby(doglevel).cumcount()


In [12]:

# Average Speed until prev race
df_base_sp['speed_prev'] = df_base_sp.groupby(doglevel)['speed'].shift(1)

df_base_sp['speed_avg'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).mean().reset_index(0,drop=True)
df_base_sp['speed_min'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).min().reset_index(0,drop=True)
df_base_sp['speed_max'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)

df_base_sp['speed_range'] = df_base_sp['speed_max'] - df_base_sp['speed_min']

df_base_sp['speed_std'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)

print(datetime.now())
      

2022-12-03 04:04:41.683781


In [15]:

# ADDITIONAL VARIABLES
grouplevel = ['@id_DOG','x_dist_gp']

# GROUPED X_DIST 
df_base_sp['x_dist_gp'] = df_base_sp['x_dist'].apply(lambda x : 300 if x < 330 else 350 if x < 380 else 400 if x < 420 else 450 if x < 485 else 500 if x < 550 else 600 if x < 650 else 700 if x < 750 else -999)
df_base_sp['x_runtm_gp'] = ( df_base_sp['y_runtime'] / df_base_sp['x_dist'] ) * df_base_sp['x_dist_gp']

df_base_sp['x_rtgp_prev'] = df_base_sp.groupby(grouplevel)['x_runtm_gp'].shift(1)

#df_base_sp['x_rtgp_avg'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().reset_index(drop=True)
df_base_sp['x_rtgp_avg'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().values

#df_base_sp['x_rtgp_min'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().reset_index(drop=True)
df_base_sp['x_rtgp_min'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().values

#df_base_sp['x_rtgp_max'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)
df_base_sp['x_rtgp_max'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().values

df_base_sp['x_rtgp_range'] = df_base_sp['x_rtgp_max'] - df_base_sp['x_rtgp_min']

#df_base_sp['x_rtgp_std'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)
df_base_sp['x_rtgp_std'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().values

print(datetime.now())


2022-12-03 04:09:04.465595


In [16]:

#df_base_sp.head(25)

#df_base_sp[ df_base_sp['@id_DOG'].isin([108390981,108391383,108391431]) ].head(25)

df_base_sp[ df_base_sp['x_runtm_gp'].isna()].head()



,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,...,date,Event_Dt,ft_sp,x_num_dogs,position,y_margin,y_pos,x_dist,speed,f_1,x_races_tot,speed_prev,speed_avg,speed_min,speed_max,speed_range,speed_std,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std


In [17]:

print(df_base_sp.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std']


In [18]:

#temp = df_base_sp.groupby('x_dist').agg( {'@id_RACE':'nunique'} ).reset_index()
#temp.to_csv(f'{fileloc}/race_distance.csv', index = False)
#temp.sort_values(by = 'index').head()


In [20]:

print( len(df_base_sp['@id_DOG'].unique().tolist()) )

print(df_base_sp['speed'].isna().sum())
print(df_base_sp['speed_prev'].isna().sum())


26806
0
26806


In [21]:

print(pd.DataFrame(df_base_sp.dtypes).transpose())


  @id_DOG   Place DogName      Box    Rug   Weight StartPrice Handicap  Margin1  Margin2     PIR Checks Comments SplitMargin y_runtime Prizemoney RaceId TrainerId TrainerName @id_RACE RaceNum  \
0   int64  object  object  float64  int64  float64     object  float64  float64  float64  object  int64   object     float64   float64    float64  int64     int64      object    int64   int64   

  RaceName RaceTime Distance RaceGrade  ...    date Event_Dt    ft_sp x_num_dogs position y_margin  y_pos x_dist    speed    f_1 x_races_tot speed_prev speed_avg speed_min speed_max speed_range  \
0   object   object   object    object  ...  object   object  float64      int64    int64  float64  int64  int64  float64  int64       int64    float64   float64   float64   float64     float64   

  speed_std x_dist_gp x_runtm_gp x_rtgp_prev x_rtgp_avg x_rtgp_min x_rtgp_max x_rtgp_range x_rtgp_std  
0   float64     int64    float64     float64    float64    float64    float64      float64    float64  

[1 ro

In [22]:

print(pd.DataFrame(df_base_sp.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  y_runtime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    588017        0    81676  78666       0    284915        98691          0      183606       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  ...  date  Event_Dt  ft_sp  x_num_dogs  position  y_margin  y_pos  x_dist  speed  f_1  x_races_tot  speed_prev  speed_avg  speed_min  speed_max  \
0        15         0         0          0  ...     0         0      0           0         0         0      0       0      0    0            0       26806      26806      26806      26806   

   speed_range  speed_std  x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg  x_rtgp_min  x_rtgp_max  x_rtgp_range  x_rtgp_std  
0        26806      52202          0           0        76823       76823       76823     


### C) Treat Missing Values


In [23]:

# All Columns Realted to Average Speed

df_base_sp['x_no_prev_races'] = df_base_sp['speed_prev'].apply( lambda x : 1 if pd.isna(x) else 0)

print( df_base_sp['x_no_prev_races'].value_counts() )

df_base_sp['x_speed_prev'] = df_base_sp['speed_prev'].fillna(0)
df_base_sp['x_speed_avg'] = df_base_sp['speed_avg'].fillna(0)
df_base_sp['x_speed_min'] = df_base_sp['speed_min'].fillna(0)
df_base_sp['x_speed_max'] = df_base_sp['speed_max'].fillna(0)

df_base_sp['x_speed_range'] = df_base_sp['speed_range'].fillna(-1)

df_base_sp['x_speed_std'] = df_base_sp['speed_std'].fillna(0)
df_base_sp['x_speed_std'] = df_base_sp.apply(lambda x : -1 if x.x_no_prev_races == 1 else x.x_speed_std, axis = 1  )

print(datetime.now())


0    561211
1     26806
Name: x_no_prev_races, dtype: int64
2022-12-03 04:13:51.209358


In [24]:

# All Columns Related to RunTime in Group

df_base_sp['x_no_prev_r_in_dist'] = df_base_sp['x_rtgp_prev'].apply( lambda x : 1 if pd.isna(x) else 0)

print( df_base_sp['x_no_prev_r_in_dist'].value_counts() )

print(datetime.now())


0    511194
1     76823
Name: x_no_prev_r_in_dist, dtype: int64
2022-12-03 04:13:51.497936


In [25]:

rtgp_mvt = df_base_sp.groupby('@id_RACE').agg({ 'x_rtgp_prev':'mean', 'x_rtgp_avg':'mean', 'x_rtgp_min':'mean', 'x_rtgp_max':'mean', 'x_rtgp_range':'mean', 'x_rtgp_std':'mean'})\
            .reset_index()\
            .rename(columns = {'x_rtgp_prev':'mvt_rtgp_prev', 'x_rtgp_avg':'mvt_rtgp_avg', 'x_rtgp_min':'mvt_rtgp_min', 'x_rtgp_max':'mvt_rtgp_max', 'x_rtgp_range':'mvt_rtgp_range', 'x_rtgp_std':'mvt_rtgp_std'})

print(pd.DataFrame(rtgp_mvt.isna().sum()).transpose())


   @id_RACE  mvt_rtgp_prev  mvt_rtgp_avg  mvt_rtgp_min  mvt_rtgp_max  mvt_rtgp_range  mvt_rtgp_std
0         0            839           591           591           591             591          1642


In [26]:

print(pd.DataFrame(df_base_sp.isna().sum()).transpose())
print()
print()
print()

print(df_base_sp.shape)
df.ft_mvt = df_base_sp.merge(rtgp_mvt, on = '@id_RACE', how = 'left')
print(df.ft_mvt.shape)

df.ft_mvt['x_rtgp_prev'].fillna(df.ft_mvt['mvt_rtgp_prev'], inplace=True)

df.ft_mvt['x_rtgp_avg'].fillna(df.ft_mvt['mvt_rtgp_avg'], inplace=True)
df.ft_mvt['x_rtgp_min'].fillna(df.ft_mvt['mvt_rtgp_min'], inplace=True)
df.ft_mvt['x_rtgp_max'].fillna(df.ft_mvt['mvt_rtgp_max'], inplace=True)

df.ft_mvt['x_rtgp_range'].fillna(df.ft_mvt['mvt_rtgp_range'], inplace=True)
df.ft_mvt['x_rtgp_std'].fillna(df.ft_mvt['mvt_rtgp_std'], inplace=True)

print()
print()
print()
print(pd.DataFrame(df.ft_mvt.isna().sum()).transpose())


# MIGHT NEED TO CHECK THIS AGAIN
df.ft_mvt['x_rtgp_prev'].fillna(0, inplace=True)

df.ft_mvt['x_rtgp_avg'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_min'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_max'].fillna(0, inplace=True)

df.ft_mvt['x_rtgp_range'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_std'].fillna(0, inplace=True)

print()
print()
print()
print(pd.DataFrame(df.ft_mvt.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  y_runtime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    588017        0    81676  78666       0    284915        98691          0      183606       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  ...  speed  f_1  x_races_tot  speed_prev  speed_avg  speed_min  speed_max  speed_range  speed_std  x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg  \
0        15         0         0          0  ...      0    0            0       26806      26806      26806      26806        26806      52202          0           0        76823       76823   

   x_rtgp_min  x_rtgp_max  x_rtgp_range  x_rtgp_std  x_no_prev_races  x_speed_prev  x_speed_avg  x_speed_min  x_speed_max  x_speed_range  x_speed_std  x_no_prev_r_in_dist  
0       76823       76823         76823

In [27]:

#df.ft_mvt[ df.ft_mvt.mvt_rtgp_prev.isna() | df.ft_mvt.x_rtgp_prev.isna()  ].head(25)


df.ft_mvt[ df.ft_mvt['@id_RACE'].isin([626271182, 626218628, 626240593])].sort_values(by = ['@id_RACE','Place']).head(25)


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,y_runtime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,...,speed_max,speed_range,speed_std,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist,mvt_rtgp_prev,mvt_rtgp_avg,mvt_rtgp_min,mvt_rtgp_max,mvt_rtgp_range,mvt_rtgp_std
16687,251076212,1,BURN OUT BOY,5.0,5,33.2,$2.60F,NaN,1.00,NaN,211,0,M21c1 Sw,6.62,22.23,1100.0,626218628,68894,G Price,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.400000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
178595,415996356,2,SCARY FUN,3.0,3,27.2,$4.00,NaN,1.00,1.14,332,0,Mc3c3c2,6.77,22.31,330.0,626218628,112106,L Isaac,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.484211,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
31489,295446742,3,METRE EATER,2.0,2,33.1,$9.90,NaN,4.25,3.00,744,0,Sc7c44,6.88,22.52,165.0,626218628,72510,B McPherson,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.705263,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
7157,219044234,4,REPEAT SET,8.0,8,30.4,$12.00,NaN,5.50,1.43,656,0,M65w6,6.84,22.62,55.0,626218628,127397,C Bainbridge,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.810526,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
14680,246692403,5,KABARANGI,1.0,1,24.1,$40.60,NaN,5.75,0.29,565,0,Q5c65,6.84,22.64,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.831579,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
5,108391383,6,SIMPLY GIFTED,6.0,6,24.8,$46.50,NaN,7.25,1.43,877,0,Sc8c77,6.93,22.74,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.936842,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
10823,231587263,7,ABBEY SIOBHAN,4.0,4,30.8,$5.60,NaN,9.00,1.71,123,0,Q12cc3,6.59,22.86,0.0,626218628,102996,M Stewart,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,24.063158,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
28025,289026900,8,CROWN BENJI,7.0,9,32.4,$6.80,NaN,18.25,9.29,488,0,M48to8 I-21,6.82,23.51,0.0,626218628,83951,R Noden,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,24.747368,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
219348,434800047,1,IVY CONNIVING,8.0,8,28.2,$2.20F,NaN,8.75,NaN,111,0,111,8.20,22.55,1600.0,626240593,108360,S Sultana,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,NaN,NaN,NaN,400,22.550000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
17046,253088672,2,SHIMMY SPARKLES,3.0,3,27.0,$19.00,NaN,8.75,8.71,544,0,544,8.50,23.16,500.0,626240593,114260,L Azzopardi,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,NaN,NaN,NaN,400,23.160000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:

print(df.ft_mvt.describe())


            @id_DOG            Box            Rug         Weight  Handicap        Margin1        Margin2         Checks    SplitMargin      y_runtime      Prizemoney        RaceId      TrainerId  \
count  5.880170e+05  588017.000000  588017.000000  588016.000000       0.0  588017.000000  506341.000000  588017.000000  489326.000000  588017.000000   404411.000000  5.880170e+05  588017.000000   
mean   4.619574e+08       4.477804       4.584839      29.925088       NaN       6.452314       2.061646       0.526102       7.284412      24.391543      426.354001  7.386968e+08  110234.634871   
std    1.145309e+08       2.344385       2.432500       3.321114       NaN       8.736779       7.862522       1.582459       2.515969       4.945432     2908.677114  6.357945e+07   77018.376912   
min    1.083910e+08       1.000000       1.000000      18.700000       NaN       0.000000       0.000000       0.000000       0.000000      10.000000        0.000000  6.238619e+08    6418.000000   
25%    4.0

In [29]:

print( df.ft_mvt.columns.values.tolist() )


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'y_runtime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'y_margin', 'y_pos', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist', 'mvt_rtgp_prev', 'mvt_rtgp_avg', 'mvt_rtgp_min', 'mvt_rtgp_max', 'mvt_rtgp_range', 'mvt_rtgp_std']


In [30]:

print(datetime.now().date())


2022-12-03


In [32]:

# 'TrainerName', 'Track' have been dummified
cols_for_drop = ['Box','Rug','Weight','Handicap','Margin1','Margin2','PIR','Checks','Comments','SplitMargin','Prizemoney','RaceId','TrainerId','RaceTime','Distance','RaceGrade','date', 'f_1','speed',\
                'Place', 'StartPrice'\
                ,'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std'\
                , 'DogName', 'TrainerName', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'position','y_runtime'\
                , 'x_dist','ft_sp', 'x_dist', 'x_num_dogs'\
                , 'mvt_rtgp_prev', 'mvt_rtgp_avg', 'mvt_rtgp_min' ,'mvt_rtgp_max' ,'mvt_rtgp_range', 'mvt_rtgp_std'\
                , 'y_margin','y_pos']

## SHOULD BE THE LAST STEP OR a Separate Code
#df.ft1 = df.ft.copy()
exp_ds = df.ft_mvt.drop(columns = cols_for_drop)

# removed the Categorical columns from the original datast, adds the dummy variables, creates a new dataset
print(exp_ds.columns)


Index(['@id_DOG', '@id_RACE', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev',
       'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist'],
      dtype='object')


In [33]:

exp_ds.head()


,@id_DOG,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
0,108390981,626271182,0,450,26.313053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,629797242,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,629797513,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,0,17121.212121,17111.495202,17101.778282,17121.212121,19.433839,13.741799,0
3,108390981,638810808,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,0,17251.908397,17158.299600,17101.778282,17251.908397,150.130115,81.647864,0
4,108390981,643554706,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,0,17069.486405,17136.096301,17069.486405,17251.908397,182.421992,80.101155,0


### EXPORTING THE SPEED PROFILE csv

In [34]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']
#'Event_Dt', 

cols_for_drop = []

todaydt = datetime.now().date()
exp_fname = f'dataprep_speed_profile_{todaydt}.csv'


In [35]:

print(pd.DataFrame(exp_ds.isna().sum()).transpose())


   @id_DOG  @id_RACE  x_races_tot  x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg  x_rtgp_min  x_rtgp_max  x_rtgp_range  x_rtgp_std  x_no_prev_races  x_speed_prev  x_speed_avg  x_speed_min  \
0        0         0            0          0           0            0           0           0           0             0           0                0             0            0            0   

   x_speed_max  x_speed_range  x_speed_std  x_no_prev_r_in_dist  
0            0              0            0                    0  


In [36]:
print(exp_ds.shape)

(588017, 19)


In [37]:

print(exp_ds.columns.values.tolist())


['@id_DOG', '@id_RACE', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist']


In [38]:

exp_ds.head()


,@id_DOG,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
0,108390981,626271182,0,450,26.313053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,629797242,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,629797513,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,0,17121.212121,17111.495202,17101.778282,17121.212121,19.433839,13.741799,0
3,108390981,638810808,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,0,17251.908397,17158.299600,17101.778282,17251.908397,150.130115,81.647864,0
4,108390981,643554706,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,0,17069.486405,17136.096301,17069.486405,17251.908397,182.421992,80.101155,0


In [39]:

exp_ds.describe()


,@id_DOG,@id_RACE,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
count,5.880170e+05,5.880170e+05,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000,588017.000000
mean,4.619574e+08,7.386968e+08,18.263215,415.410534,24.318600,24.064572,24.028276,23.687895,24.483122,0.795227,0.309839,0.045587,16322.904980,16320.410704,15829.515646,16723.476525,893.915292,257.016920,0.130648
std,1.145309e+08,6.357945e+07,15.966707,78.101987,4.646335,5.184201,4.872720,4.833994,4.945262,0.700004,0.198714,0.208588,3591.717846,3581.260613,3493.665477,3696.710772,693.916933,198.070187,0.337015
min,1.083910e+08,6.238619e+08,0.000000,-999.000000,-62.082592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000
25%,4.005005e+08,6.854128e+08,6.000000,350.000000,20.730000,20.620000,20.660606,20.392647,21.100000,0.366732,0.214779,0.000000,16802.443992,16880.406840,16318.537859,17232.704403,553.198838,185.586364,0.000000
50%,4.569441e+08,7.402821e+08,14.000000,400.000000,23.688312,23.650000,23.584416,23.239583,24.110000,0.745033,0.288768,0.000000,17087.912088,17104.110824,16611.295681,17513.134851,867.118652,248.107160,0.000000
75%,5.405822e+08,7.922435e+08,27.000000,450.000000,27.156613,27.031323,26.722770,26.403261,27.261021,1.104545,0.374250,0.000000,17350.157729,17289.960631,16849.199663,17746.228926,1161.520074,316.833037,0.000000
max,2.145360e+09,8.483914e+08,124.000000,700.000000,384.582524,61.747368,46.248951,46.248951,61.747368,39.167368,10.132302,1.000000,40200.000000,40200.000000,40200.000000,40200.000000,23826.557771,16614.431948,1.000000


In [40]:

exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)


In [ ]:

print('THE END')
